In [45]:
import pandas as pd
import numpy as np
import numpy.linalg as npl
import seaborn as sns
import matplotlib.pyplot as plt

In [46]:
trainDf = pd.read_csv('../data/train.csv', index_col=0)
testDf = pd.read_csv('../data/test.csv', index_col=0)

In [47]:
trainDf.head()

,BertzCT,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3v,Chi4n,EState_VSA1,EState_VSA2,...,SlogP_VSA3,VSA_EState9,fr_COO,fr_COO2,EC1,EC2,EC3,EC4,EC5,EC6
id,,,,,,,,,,,,,,,,,,,,,
0,323.390782,9.879918,5.875576,5.875576,4.304757,4.304757,2.754513,1.749203,0.000000,11.938294,...,4.794537,35.527357,0,0,1,1,0,0,0,0
1,273.723798,7.259037,4.441467,5.834958,3.285046,4.485235,2.201375,1.289775,45.135471,0.000000,...,13.825658,44.707310,0,0,0,1,1,0,0,0
2,521.643822,10.911303,8.527859,11.050864,6.665291,9.519706,5.824822,1.770579,15.645394,6.606882,...,17.964475,45.660120,0,0,1,1,0,0,1,0
3,567.431166,12.453343,7.089119,12.833709,6.478023,10.978151,7.914542,3.067181,95.639554,0.000000,...,31.961948,87.509997,0,0,1,1,0,0,0,0
4,112.770735,4.414719,2.866236,2.866236,1.875634,1.875634,1.036450,0.727664,17.980451,12.841643,...,9.589074,33.333333,2,2,1,0,1,1,1,0


## train

In [48]:
for count in trainDf.count():
    if count != trainDf.shape[0]:
        print('Nan exist')

In [49]:
set(trainDf.columns) - set(testDf.columns)

{'EC1', 'EC2', 'EC3', 'EC4', 'EC5', 'EC6'}

In [50]:
trainDf.drop([f'EC{i}'for i in range(3,7)], axis=1, inplace=True)

In [51]:
trainDf.describe()

,BertzCT,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3v,Chi4n,EState_VSA1,EState_VSA2,...,PEOE_VSA7,PEOE_VSA8,SMR_VSA10,SMR_VSA5,SlogP_VSA3,VSA_EState9,fr_COO,fr_COO2,EC1,EC2
count,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,...,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000
mean,515.153604,9.135189,5.854307,6.738497,4.432570,5.253221,3.418749,1.773472,29.202823,10.435316,...,11.318811,6.704487,15.666766,31.066423,13.636941,49.309959,0.458215,0.459226,0.667745,0.798962
std,542.456370,6.819989,4.647064,5.866444,3.760516,4.925065,3.436208,1.865898,31.728679,13.651843,...,20.169745,10.865415,18.080208,33.896638,14.598554,29.174824,0.667948,0.668111,0.471038,0.400790
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,-5.430556,0.000000,0.000000,0.000000,0.000000
25%,149.103601,4.680739,2.844556,2.932842,1.949719,2.034468,1.160763,0.503897,5.969305,0.000000,...,0.000000,0.000000,5.969305,6.420822,4.794537,30.000000,0.000000,0.000000,0.000000,1.000000
50%,290.987941,6.485270,4.052701,4.392859,2.970427,3.242775,1.948613,1.073261,17.353601,6.420822,...,0.000000,0.000000,11.752550,20.075376,9.589074,41.666667,0.000000,0.000000,1.000000,1.000000
75%,652.652585,11.170477,7.486791,8.527859,5.788793,6.609350,4.502070,2.534281,44.876559,12.841643,...,13.847474,6.923737,17.721856,42.727765,14.912664,56.090650,1.000000,1.000000,1.000000,1.000000
max,4069.959780,69.551167,50.174588,53.431954,32.195368,34.579313,22.880836,16.072810,363.705954,99.936429,...,211.501279,100.348416,80.742293,492.729739,115.406157,384.450519,8.000000,8.000000,1.000000,1.000000


In [52]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaledTrainDf = pd.DataFrame(scaler.fit_transform(trainDf.drop(['EC1','EC2'],axis=1)), columns=trainDf.drop(['EC1','EC2'],axis=1).columns, index=trainDf.drop(['EC1','EC2'],axis=1).index)

In [53]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.85)
pcaTrain = pca.fit_transform(scaledTrainDf)
columns = [f'PC{i}' for i in range(pca.n_components_)]

In [54]:
pcaTrainDf = pd.DataFrame(pcaTrain, columns=columns, index=trainDf.index)
pcaTrainDf = pcaTrainDf.merge(trainDf[['EC1','EC2']],right_index=True,left_index=True)

In [55]:
pcaTrainDf

,PC0,PC1,PC2,PC3,PC4,PC5,PC6,EC1,EC2
id,,,,,,,,,
0,-1.316896,0.010559,0.172449,-1.206987,1.323193,-0.096639,1.987939,1,1
1,-0.602016,-0.071304,-1.016605,-1.955152,-0.030156,-0.100062,-1.414728,0,1
2,1.329880,0.072148,1.350675,-0.858966,1.265800,0.005934,-0.793232,1,1
3,3.846470,0.043175,-1.611639,-1.882988,0.169146,-0.230597,-2.009355,1,1
4,-2.620696,-0.199726,-1.012035,2.801416,0.607764,0.029445,-0.329241,1,0
...,...,...,...,...,...,...,...,...,...
14833,1.538039,-0.137586,-0.732492,-0.968503,-0.505130,0.006628,-0.138886,1,1
14834,-4.583786,-0.112315,1.017409,-1.092308,-0.982864,0.128770,0.473880,0,1
14835,0.599809,-0.150772,0.237708,0.039884,-1.974062,-0.063152,-0.031873,1,1


In [92]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate

X = pcaTrainDf.drop(['EC1','EC2'], axis=1)
y = pcaTrainDf['EC1']
lr = LogisticRegression()
cv = cross_validate(lr,X,y,)
cv['test_score']

array([0.67991914, 0.67688679, 0.6856469 , 0.6892484 , 0.67576677])

In [93]:
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN()
lr1 = LogisticRegression()
model1 = lr1.fit(X,pcaTrainDf['EC1'])
lr2 = LogisticRegression()
X_re, y_re = smote_enn.fit_resample(X,pcaTrainDf['EC2'])
model2 = lr2.fit(X_re, y_re)

## test

In [94]:
scaledTestDf = pd.DataFrame(scaler.transform(testDf), columns=testDf.columns, index=testDf.index)
pcaTest = pca.transform(scaledTestDf)
columns = [f'PC{i}' for i in range(pca.n_components_)]
pcaTestDf = pd.DataFrame(pcaTest, columns=columns, index=testDf.index)

In [95]:
pcaTestDf

,PC0,PC1,PC2,PC3,PC4,PC5,PC6
id,,,,,,,
14838,-0.974964,-0.084144,-1.738369,0.013456,0.400114,-0.040072,-1.162768
14839,0.828864,0.373184,4.512659,1.183287,0.236587,-0.056466,0.211633
14840,-3.303044,-0.126413,-0.618430,0.888992,-0.297298,0.170807,0.366505
14841,-2.398186,-0.128536,0.253518,-1.225066,0.867021,-0.106140,1.228102
14842,9.766206,-0.038719,-0.829720,-0.106847,-2.116485,0.238749,1.157570
...,...,...,...,...,...,...,...
24726,-2.639052,-0.257550,-1.801444,2.477261,0.392997,-0.017482,-0.476838
24727,-0.131210,-0.060512,2.015513,0.767981,-1.468214,0.016525,1.228724
24728,-1.227965,0.119874,-0.433521,-1.593615,-0.236472,-0.113853,-1.030054


In [96]:
EC1 = model1.predict_proba(pcaTestDf)[:,1]
EC2 = model2.predict_proba(pcaTestDf)[:,1]

In [97]:
submissionFile = pd.DataFrame({'EC1':EC1, 'EC2':EC2}, index=testDf.index)

In [98]:
submissionFile.to_csv('testSubmission2.csv')